# Jython Execution and Batch Runs

This notebook demonstrates how to automate HEC-HMS simulations using Jython scripts.

**Key capabilities covered:**
- Discover installed HMS versions
- Extract example projects from HMS installations
- Generate and execute Jython automation scripts
- Configure JVM memory for large models
- Optional: Batch execution of multiple runs

**Version Compatibility:**

| HMS Version | Support | Notes |
|-------------|---------|-------|
| **4.4.1+** | Full | Direct Java execution with memory control |
| **4.0 - 4.3** | Limited | Legacy classpath (use GUI) |
| **3.x** | None | No Jython scripting (requires `python2_compatible=True` for M3 models) |

In [ ]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

## Setup

In [ ]:
import re
import time
from pathlib import Path

from hms_commander import HmsExamples, HmsJython, __version__

print(f"hms-commander v{__version__}")

## 1. Discover Installed HMS Versions

`HmsExamples` automatically scans standard installation paths for HEC-HMS.

In [ ]:
# List all installed versions (sorted newest first)
versions = HmsExamples.list_versions()
print(f"Found {len(versions)} HMS versions installed:")
print(versions)

In [ ]:
# Get detailed info about installations
install_info = HmsExamples.detect_installed_versions()
for version, path in sorted(install_info.items(), 
                            key=lambda x: [int(p) for p in x[0].split('.')], 
                            reverse=True)[:5]:
    print(f"HMS {version}: {path}")

## 2. Select Version for Testing

We'll use the newest HMS 4.x version available (4.4.1 or later required for scripting).

In [ ]:
# Filter to supported versions (4.4.1+)
def version_tuple(v):
    return tuple(int(x) for x in v.split('.'))

hms4_versions = [v for v in versions if v.startswith('4.')]
supported_versions = [v for v in hms4_versions if version_tuple(v) >= (4, 4)]

if not supported_versions:
    print("WARNING: No supported HMS versions found (need 4.4.1 or later)")
    selected_version = None
else:
    # Use the newest available version
    selected_version = sorted(supported_versions, key=version_tuple, reverse=True)[0]
    print(f"Selected version: HMS {selected_version}")
    print(f"All supported versions: {supported_versions}")

## 3. List Available Example Projects

Each HMS version includes example projects in its `samples.zip` file.

In [ ]:
# List all projects across all versions
all_projects = HmsExamples.list_projects()

if selected_version:
    projects = all_projects.get(selected_version, [])
    print(f"Projects in HMS {selected_version}: {projects}")
else:
    print("Example projects by version:")
    for version in list(all_projects.keys())[:5]:
        print(f"  HMS {version}: {all_projects[version]}")

In [ ]:
# Get detailed info about the tifton project
info = HmsExamples.get_project_info('tifton')
print("Tifton Project Info:")
print(f"  HMS File: {info['hms_file']}")
print(f"  Basin Models: {info['basin_models']}")
print(f"  Met Models: {info['met_models']}")
print(f"  Control Specs: {info['control_specs']}")
print(f"  Has DSS Data: {info['has_dss']}")

## 4. Extract Project and Select Run

In [ ]:
# Extract tifton project for the selected version
if selected_version:
    # Output directory for extracted projects
    output_base = Path.cwd() / 'hms_example_projects' / 'jython_execution'
    output_base.mkdir(parents=True, exist_ok=True)
    
    print(f"Extracting tifton for HMS {selected_version}...")
    project_path = HmsExamples.extract_project(
        'tifton',
        version=selected_version,
        output_path=output_base
    )
    
    # Parse run name from .run file
    run_file = project_path / 'tifton.run'
    run_content = run_file.read_text()
    run_match = re.search(r'^Run:\s*(.+)$', run_content, re.MULTILINE)
    run_name = run_match.group(1).strip() if run_match else 'Unknown'
    
    # Get HMS executable path
    hms_exe = HmsExamples.get_hms_exe(selected_version)
    
    print(f"  Path: {project_path}")
    print(f"  Run Name: '{run_name}'")
    print(f"  HMS Exe: {hms_exe}")
else:
    print("No supported HMS version available for extraction")

## 5. Generate Jython Script

HEC-HMS 4.x can be automated via Jython scripts using the `JythonHms` API.

In [ ]:
# Generate a compute script
if selected_version and 'project_path' in dir():
    script = HmsJython.generate_compute_script(
        project_path=project_path,
        run_name=run_name
    )
    
    print(f"Generated Jython script for HMS {selected_version}:")
    print("-" * 50)
    print(script)
else:
    print("No project extracted - cannot generate script")

## 6. Execute Jython Script

The direct Java approach bypasses the `HEC-HMS.cmd` batch file entirely:
- **Avoids path quoting bugs** in HMS 4.4-4.11 batch files
- **Enables JVM memory control** (`-Xmx` for large models)
- **Automatic 32-bit detection** (caps memory for legacy installations)

In [ ]:
# Execute the simulation
if selected_version and 'script' in dir():
    print(f"Executing HMS {selected_version}: '{run_name}'")
    print("=" * 60)
    
    start_time = time.time()
    success, stdout, stderr = HmsJython.execute_script(
        script_content=script,
        hms_exe_path=hms_exe,
        working_dir=project_path,
        timeout=300,       # 5 minute timeout
        max_memory="2G",   # JVM max heap (increase for large models)
        initial_memory="128M"  # JVM initial heap
    )
    elapsed = time.time() - start_time
    
    # Check for success indicators
    actual_success = ('Computation completed' in stdout) or (success and 'Error' not in stderr)
    
    status = "SUCCESS" if actual_success else "FAILED"
    print(f"Result: {status} ({elapsed:.1f}s)")
    
    if 'Project opened' in stdout:
        print("  - Project opened successfully")
    if 'Computation completed' in stdout:
        print("  - Computation completed")
    
    if not actual_success and stderr:
        print(f"Error: {stderr[:500]}")
else:
    print("No script generated - cannot execute")

## 7. Verify DSS Output

In [ ]:
# Check for DSS output file
if 'project_path' in dir():
    dss_files = list(project_path.glob('*.dss'))
    output_dss = [f for f in dss_files if 'simulation' in f.name.lower() 
                  or f.stem == run_name.replace(' ', '_')]
    
    if output_dss:
        dss_file = output_dss[0]
        file_size_kb = dss_file.stat().st_size / 1024
        print(f"DSS output file: {dss_file.name}")
        print(f"  Size: {file_size_kb:,.1f} KB")
        print(f"  [OK] Simulation produced DSS output")
    else:
        print("DSS output files found:")
        for f in dss_files:
            print(f"  - {f.name}")

## Summary

This notebook demonstrated single-project Jython execution:

| Feature | Method | Description |
|---------|--------|-------------|
| Version discovery | `HmsExamples.list_versions()` | Find installed HMS versions |
| Project extraction | `HmsExamples.extract_project()` | Get example projects |
| Executable path | `HmsExamples.get_hms_exe()` | Get HMS executable |
| Script generation | `HmsJython.generate_compute_script()` | Create Jython script |
| Script execution | `HmsJython.execute_script()` | Run via direct Java |

### Memory Configuration for Large Models

```python
# Default (4GB) - small/medium models
success, stdout, stderr = HmsJython.execute_script(script, hms_exe)

# Large model (8GB)
success, stdout, stderr = HmsJython.execute_script(
    script, hms_exe, max_memory="8G"
)

# Very large model (16GB) with custom GC
success, stdout, stderr = HmsJython.execute_script(
    script, hms_exe, 
    max_memory="16G",
    additional_java_opts=["-XX:+UseG1GC"]
)
```

---

## Appendix: Batch Execution (Optional)

The following cells demonstrate how to execute all runs across all example projects for a single HMS version. This is useful for validation testing.

In [ ]:
# Helper function: Parse runs from project
def get_runs_from_project(project_path: Path) -> list:
    """
    Parse all run names from the .run file in a project.
    
    Returns:
        List of run names
    """
    runs = []
    run_files = list(project_path.glob('*.run'))
    if not run_files:
        return runs
    
    run_file = run_files[0]
    content = run_file.read_text(encoding='utf-8', errors='ignore')
    
    for match in re.finditer(r'^Run:\s*(.+)$', content, re.MULTILINE):
        run_name = match.group(1).strip()
        if run_name:
            runs.append(run_name)
    
    return runs

print("Helper function defined.")

In [ ]:
# Batch execution configuration
BATCH_VERSION = selected_version  # Use the version from above
BATCH_OUTPUT = Path.cwd() / 'hms_example_projects' / 'batch_run_all'

# Skip batch execution by default (set to True to run)
RUN_BATCH = False

if RUN_BATCH and BATCH_VERSION:
    BATCH_OUTPUT.mkdir(parents=True, exist_ok=True)
    batch_hms_exe = HmsExamples.get_hms_exe(BATCH_VERSION)
    
    print(f"Batch execution configured for HMS {BATCH_VERSION}")
    print(f"Output: {BATCH_OUTPUT}")
else:
    print("Batch execution skipped (set RUN_BATCH = True to enable)")

In [ ]:
# Extract all projects and discover runs
if RUN_BATCH and BATCH_VERSION:
    projects = all_projects.get(BATCH_VERSION, [])
    project_info = {}
    
    for project_name in projects:
        print(f"Extracting: {project_name}")
        try:
            proj_path = HmsExamples.extract_project(
                project_name,
                version=BATCH_VERSION,
                output_path=BATCH_OUTPUT / project_name
            )
            runs = get_runs_from_project(proj_path)
            project_info[project_name] = {'path': proj_path, 'runs': runs}
            print(f"  Runs: {runs}")
        except Exception as e:
            print(f"  ERROR: {e}")
    
    total_runs = sum(len(p['runs']) for p in project_info.values())
    print(f"\nTotal: {len(project_info)} projects, {total_runs} runs")

In [ ]:
# Execute all runs
if RUN_BATCH and BATCH_VERSION and 'project_info' in dir():
    import pandas as pd
    
    results = []
    total_runs = sum(len(p['runs']) for p in project_info.values() if p.get('path'))
    run_count = 0
    
    print(f"Executing {total_runs} runs...")
    print("=" * 70)
    
    for project_name, info in project_info.items():
        if not info.get('path'):
            continue
        
        for run_name in info['runs']:
            run_count += 1
            print(f"[{run_count}/{total_runs}] {project_name} / {run_name}")
            
            script = HmsJython.generate_compute_script(
                project_path=info['path'],
                run_name=run_name,
                save_project=True
            )
            
            start_time = time.time()
            try:
                success, stdout, stderr = HmsJython.execute_script(
                    script_content=script,
                    hms_exe_path=batch_hms_exe,
                    working_dir=info['path'],
                    timeout=300,
                    max_memory="4G"
                )
                elapsed = time.time() - start_time
                actual_success = ('Computation completed' in stdout) or (success and 'Error' not in stderr)
                error_msg = stderr[:200] if not actual_success and stderr else None
            except Exception as e:
                elapsed = time.time() - start_time
                actual_success = False
                error_msg = str(e)[:200]
            
            results.append({
                'project': project_name,
                'run': run_name,
                'success': actual_success,
                'time': elapsed,
                'error': error_msg
            })
            
            status = "SUCCESS" if actual_success else "FAILED"
            print(f"  {status} ({elapsed:.1f}s)")
    
    # Summary
    df = pd.DataFrame(results)
    total = len(df)
    passed = df['success'].sum()
    
    print(f"\n{'=' * 70}")
    print(f"BATCH EXECUTION SUMMARY")
    print(f"{'=' * 70}")
    print(f"Total Runs:  {total}")
    print(f"Passed:      {passed} ({100*passed/total:.0f}%)")
    print(f"Failed:      {total - passed}")
    print(f"Total Time:  {df['time'].sum():.1f}s")

## Next Steps

- **03_project_dataframes.ipynb** - Explore project data structures
- **04_file_operations.ipynb** - Read and modify HMS files
- **08_m3_models.ipynb** - Work with HCFCD M3 Model archives